In [17]:
import base64
import requests
from dotenv import load_dotenv
import os
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_core.tools import ToolException
load_dotenv()
BOOKING_API_KEY=os.getenv('BOOKING_API_KEY')
encoded_key=base64.b64encode(BOOKING_API_KEY.encode()).decode()
BASE_URL="https://api.resos.com/v1/"
headers={
    'Authorization': f'Basic {encoded_key}'
}

def get_areas():
    url=BASE_URL + "tables"
    response=requests.get(url,headers=headers)
    return response.json()

def get_area_by_name(name:str)->dict:
    areas=get_areas()
    for area in areas :
        if name == area['name']:
            return area["_id"]
    return None

class AvailableTimes(BaseModel):
    date : str =Field(description="Es una fecha en el formato YYYY-MM-DD, debe ser mayor o igual a la fecha actual")
    people : int =Field(description="Cantidad de personas que necesita el cliente" )

def available_times_at_date(date: str ,people=4,area=None):
    """Fetch the available times for booking, given a date and  a number of people"""
    url=BASE_URL + "bookingFlow/times"
    #parameters for the request
    params={
        'date' :date ,
        'people' : people
    }
    if area != None :
        params['area']=area
    #Make the request
    response=requests.get(url,headers=headers,params=params)
    if response.status_code == 200:
        results=response.json()
    else: 
        raise ToolException(f"An error has ocurred with status code  : {response.status_code} and content {response.content}")
    result=results[0]
    if len(result['unavailableTimes'] ) == 0:
        return " The whole day is available for booking"
    else : 
        return f" The available booking times for {date} are : {result['availableTimes']} "

tool_available_time=StructuredTool.from_function(
    func=available_times_at_date,
    name="available_times_at_date",
    description="Fetch the available times for booking, given a date and  a number of people",
    handle_tool_error=True
)

class createBooking(BaseModel):
    date : str =Field(description="Es una fecha en el formato YYYY-MM-DD, debe ser mayor o igual a la fecha actual")
    people : int =Field(description="Cantidad de personas que se necesitan para la reserva" )
    time : str =Field(description="Hora en la que se realizara la reserva")
    people : int=Field(description="Cantidad de personas que van a entrar en la reserva")
    phone : str=Field(description="El numero telefonico de la persona que reserva")
    email : str=Field(description="El correo electronico de la persona que hace la reserva")
    name : str=Field(description="Nombre de la persona que hace la reserva")
    area_name : str=Field(description="Nombre del area donde se va a hacer la reserva")
    duration : str=Field(description="Duracion de la reserva")
    tables : str=Field(description="Cantidad de mesas que usara el cliente")

def create_booking(date : str , time : str, people : int , phone : str ,email : str, name : str ,area_name='main', duration = 120 ,tables=1):
    """ Create a booking for the given date, time, number of people, and area if desired"""
    #Obtiene el id del area dado un nombre
    area_id=get_area_by_name(area_name)
    if area_id==None:
        return " No existe un area con ese nombre"
    url=BASE_URL+'bookings'
    params= {
        "date" : date,
        "time" : time,
        "people" : people,
        "duration" :duration,
        "guest" : {
            "name" : name,
            "phone" : phone,
            "email" : email,
        },
        "areaId" : area_id,
        "status" : "pending"
    }
    headers_post=headers
    headers_post['Content-Type']='application/json'
    response=requests.post(url,headers=headers_post,json=params)
    if response.status_code==200:
        booking_id=response.content
        return f"Reserva creada exitosamente : {response.content}"
    else : 
        raise ToolException(f"Reserva fallida debido a  {response.content}")

tool_create_booking=StructuredTool.from_function(
    func=create_booking,
    name="create_booking",
    description="Create a new booking ",
    handle_tool_error=True
)

In [18]:
SYSTEM_TEMPLATE="""
Eres un asistente virtual para un restaurante llamado Rooftop Magdalena. Tendras 2 funciones principales : 
{tools}

Obtener los horarios disponibles para hacer reservas dada una fecha, o hacer una reserva. Cuando el usuario tenga que hacer una reserva, lo primero que le vas a preguntar va a ser la fecha(formato YYYY-MM-DD) en la cual la quiere hacer y la cantidad de personas para las cuales quiere hacer la reserva. Luego validas si esa fecha esta disponible y le muestras los horarios. . Finalmente, vas a preguntarle por sus datos personales, como el nombre, el telefono y el email. y luego vas a mostrarle los datos que el ingresó. Cuando el usuario confirme, ahi si usaras la funcion de crear reserva, antes no. 
"""

In [19]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(model="gpt-4o", temperature=0.2)

In [20]:
tools=[tool_create_booking,tool_available_time]

In [21]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [22]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

In [23]:
prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                SYSTEM_TEMPLATE,
            ),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
agent= create_openai_tools_agent(chat,tools,prompt)
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [24]:
from langchain_core.runnables.history import RunnableWithMessageHistory

In [25]:
import logging
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

In [26]:
store = {}
#Gets the chat history based on the cellphone number (id)
#For production use cases, you will want to use a persistent implementation of chat message history, such as RedisChatMessageHistory
def get_session_history(session_id: str) -> BaseChatMessageHistory:

    if session_id not in store:
        logging.info(f"Creating new session history for {session_id}")
        store[session_id]=ChatMessageHistory()
        logging.info(f"Session history for {session_id} created")

    return store[session_id]

In [27]:
chain_with_message_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    )

In [28]:
chain_with_message_history.invoke({"input": "Hola, quisiera hacer una reserva.Me llamo Pablo", "tools": tools},{"configurable":{"session_id":"1234"}})



> Entering new AgentExecutor chain...
¡Hola Pablo! Con gusto te ayudo a hacer una reserva. Primero, necesito saber la fecha en la que te gustaría hacer la reserva (formato YYYY-MM-DD) y la cantidad de personas para las cuales quieres hacer la reserva.

> Finished chain.


{'input': 'Hola, quisiera hacer una reserva.Me llamo Pablo',
 'tools': [StructuredTool(name='create_booking', description="create_booking(date: str, time: str, people: int, phone: str, email: str, name: str, area_name='main', duration=120, tables=1) - Create a new booking", args_schema=<class 'pydantic.v1.main.create_bookingSchema'>, handle_tool_error=True, func=<function create_booking at 0x00000222DE6DE480>),
  StructuredTool(name='available_times_at_date', description='available_times_at_date(date: str, people=4, area=None) - Fetch the available times for booking, given a date and  a number of people', args_schema=<class 'pydantic.v1.main.available_times_at_dateSchema'>, handle_tool_error=True, func=<function available_times_at_date at 0x00000222BAF73920>)],
 'chat_history': [],
 'output': '¡Hola Pablo! Con gusto te ayudo a hacer una reserva. Primero, necesito saber la fecha en la que te gustaría hacer la reserva (formato YYYY-MM-DD) y la cantidad de personas para las cuales quieres

In [29]:
chain_with_message_history.invoke({"input": "cual era mi nombre ?", "tools": tools},{"configurable":{"session_id":"1234"}})



> Entering new AgentExecutor chain...
Tu nombre es Pablo.

> Finished chain.


{'input': 'cual era mi nombre ?',
 'tools': [StructuredTool(name='create_booking', description="create_booking(date: str, time: str, people: int, phone: str, email: str, name: str, area_name='main', duration=120, tables=1) - Create a new booking", args_schema=<class 'pydantic.v1.main.create_bookingSchema'>, handle_tool_error=True, func=<function create_booking at 0x00000222DE6DE480>),
  StructuredTool(name='available_times_at_date', description='available_times_at_date(date: str, people=4, area=None) - Fetch the available times for booking, given a date and  a number of people', args_schema=<class 'pydantic.v1.main.available_times_at_dateSchema'>, handle_tool_error=True, func=<function available_times_at_date at 0x00000222BAF73920>)],
 'chat_history': [HumanMessage(content='Hola, quisiera hacer una reserva.Me llamo Pablo'),
  AIMessage(content='¡Hola Pablo! Con gusto te ayudo a hacer una reserva. Primero, necesito saber la fecha en la que te gustaría hacer la reserva (formato YYYY-MM-D

In [30]:
chain_with_message_history.invoke({"input": "Quiero ver la disponibilidad para el 27 de mayo", "tools": tools},{"configurable":{"session_id":"1234"}})



> Entering new AgentExecutor chain...
Claro, ¿para cuántas personas te gustaría hacer la reserva?

> Finished chain.


{'input': 'Quiero ver la disponibilidad para el 27 de mayo',
 'tools': [StructuredTool(name='create_booking', description="create_booking(date: str, time: str, people: int, phone: str, email: str, name: str, area_name='main', duration=120, tables=1) - Create a new booking", args_schema=<class 'pydantic.v1.main.create_bookingSchema'>, handle_tool_error=True, func=<function create_booking at 0x00000222DE6DE480>),
  StructuredTool(name='available_times_at_date', description='available_times_at_date(date: str, people=4, area=None) - Fetch the available times for booking, given a date and  a number of people', args_schema=<class 'pydantic.v1.main.available_times_at_dateSchema'>, handle_tool_error=True, func=<function available_times_at_date at 0x00000222BAF73920>)],
 'chat_history': [HumanMessage(content='Hola, quisiera hacer una reserva.Me llamo Pablo'),
  AIMessage(content='¡Hola Pablo! Con gusto te ayudo a hacer una reserva. Primero, necesito saber la fecha en la que te gustaría hacer la

In [33]:
chain_with_message_history.invoke({"input": "Seria para 5 personas, recuerda que estamos en 2024 ?", "tools": tools},{"configurable":{"session_id":"1234"}})



> Entering new AgentExecutor chain...

Invoking: `available_times_at_date` with `{'date': '2024-05-27', 'people': 5}`


 The whole day is available for bookingEl 27 de mayo de 2024 está completamente disponible para reservas. Aquí tienes los horarios disponibles:

- Todo el día

Por favor, indícame la hora específica en la que te gustaría hacer la reserva.

> Finished chain.


{'input': 'Seria para 5 personas, recuerda que estamos en 2024 ?',
 'tools': [StructuredTool(name='create_booking', description="create_booking(date: str, time: str, people: int, phone: str, email: str, name: str, area_name='main', duration=120, tables=1) - Create a new booking", args_schema=<class 'pydantic.v1.main.create_bookingSchema'>, handle_tool_error=True, func=<function create_booking at 0x00000222DE6DE480>),
  StructuredTool(name='available_times_at_date', description='available_times_at_date(date: str, people=4, area=None) - Fetch the available times for booking, given a date and  a number of people', args_schema=<class 'pydantic.v1.main.available_times_at_dateSchema'>, handle_tool_error=True, func=<function available_times_at_date at 0x00000222BAF73920>)],
 'chat_history': [HumanMessage(content='Hola, quisiera hacer una reserva.Me llamo Pablo'),
  AIMessage(content='¡Hola Pablo! Con gusto te ayudo a hacer una reserva. Primero, necesito saber la fecha en la que te gustaría ha

In [34]:
chain_with_message_history.invoke({"input": "a las 8pm, mi nombre es Pablo Guatibonza , mi telefono es +573227077343 y mi gmail pabloaguati@gmail.com", "tools": tools},{"configurable":{"session_id":"1234"}})



> Entering new AgentExecutor chain...
Gracias por proporcionar tus datos. Aquí tienes un resumen de la información de tu reserva:

- **Fecha:** 27 de mayo de 2024
- **Hora:** 8:00 PM
- **Personas:** 5
- **Nombre:** Pablo Guatibonza
- **Teléfono:** +573227077343
- **Email:** pabloaguati@gmail.com

¿Confirmas que toda la información es correcta para proceder con la reserva?

> Finished chain.


{'input': 'a las 8pm, mi nombre es Pablo Guatibonza , mi telefono es +573227077343 y mi gmail pabloaguati@gmail.com',
 'tools': [StructuredTool(name='create_booking', description="create_booking(date: str, time: str, people: int, phone: str, email: str, name: str, area_name='main', duration=120, tables=1) - Create a new booking", args_schema=<class 'pydantic.v1.main.create_bookingSchema'>, handle_tool_error=True, func=<function create_booking at 0x00000222DE6DE480>),
  StructuredTool(name='available_times_at_date', description='available_times_at_date(date: str, people=4, area=None) - Fetch the available times for booking, given a date and  a number of people', args_schema=<class 'pydantic.v1.main.available_times_at_dateSchema'>, handle_tool_error=True, func=<function available_times_at_date at 0x00000222BAF73920>)],
 'chat_history': [HumanMessage(content='Hola, quisiera hacer una reserva.Me llamo Pablo'),
  AIMessage(content='¡Hola Pablo! Con gusto te ayudo a hacer una reserva. Primer

In [35]:
chain_with_message_history.invoke({"input": "si", "tools": tools},{"configurable":{"session_id":"1234"}})



> Entering new AgentExecutor chain...

Invoking: `create_booking` with `{'date': '2024-05-27', 'time': '20:00', 'people': 5, 'phone': '+573227077343', 'email': 'pabloaguati@gmail.com', 'name': 'Pablo Guatibonza'}`


Reserva fallida debido a  b'status not allowed [bookings.insert: status not allowed]'Parece que hubo un problema al intentar crear la reserva. El sistema devolvió un error indicando que el estado no está permitido.

Voy a intentar nuevamente.

> Finished chain.


{'input': 'si',
 'tools': [StructuredTool(name='create_booking', description="create_booking(date: str, time: str, people: int, phone: str, email: str, name: str, area_name='main', duration=120, tables=1) - Create a new booking", args_schema=<class 'pydantic.v1.main.create_bookingSchema'>, handle_tool_error=True, func=<function create_booking at 0x00000222DE6DE480>),
  StructuredTool(name='available_times_at_date', description='available_times_at_date(date: str, people=4, area=None) - Fetch the available times for booking, given a date and  a number of people', args_schema=<class 'pydantic.v1.main.available_times_at_dateSchema'>, handle_tool_error=True, func=<function available_times_at_date at 0x00000222BAF73920>)],
 'chat_history': [HumanMessage(content='Hola, quisiera hacer una reserva.Me llamo Pablo'),
  AIMessage(content='¡Hola Pablo! Con gusto te ayudo a hacer una reserva. Primero, necesito saber la fecha en la que te gustaría hacer la reserva (formato YYYY-MM-DD) y la cantidad d

In [36]:
create_booking("2024-05-30","20:00",4,"+573227077343","pabloaguati@gmail.com","pablito")

ToolException: Reserva fallida debido a  b'status not allowed [bookings.insert: status not allowed]'

In [55]:
def create_booking(date : str , time : str, people : int , phone : str ,email : str, name : str ,area_name='main', duration = 120 ,tables=1):
    """ Create a booking for the given date, time, number of people, and area if desired"""
    #Obtiene el id del area dado un nombre
    area_id=get_area_by_name(area_name)
    if area_id==None:
        return " No existe un area con ese nombre"

    url=BASE_URL+'bookings'
    params= {
        "date" : date,
        "time" : time,
        "people" : people,
        "duration" :duration,
        "guest" : {
            "name" : name,
            "phone" : phone,
            "email" : email,
        },
        "areaId" : area_id,
        "status" : "Accepted"
    }
    headers_post=headers
    headers_post['Content-Type']='application/json'

    response=requests.post(url,headers=headers_post,json=params)
    if response.status_code==200:
        booking_id=response.content
        return f"Reserva creada exitosamente : {response.content}"
    else : 
        return response
        raise ToolException(f"Reserva fallida debido a  {response.content}")

In [56]:
response=create_booking("2024-05-30","20:00",4,"+573227077343","pabloaguati@gmail.com","pablito")

In [57]:
response

'Reserva creada exitosamente : b\'"YMtMg9vMzDjSDyZf7"\''

In [48]:
response.status_code

422